# ENTSO-E Daten

Die Daten der European Network of Transmission System Operators for Electricity (ENTSO-E) werden auf dem Transparenzportal veröffentlicht.

Über eine API lassen sich die Daten, sowie Historiendaten abfragen.

Das Projekt unterteilt sich zunächst in 3 Teile:

1. Daten abfragen und speichern
2. Daten aufbereiten
3. Daten visualisieren

Referenzen:
https://energy-charts.info/
https://www.electricitymap.org/map

### 0. Begriffserklärungen / Glossar

* **Generation** beschreibt das erzeugte, also den gesamten eingespeisten Strom, ohne absorbierter Energie
* **Load/Last** beschreibt "das Verbrauchte", also das Erzeugte und Importierte minus absorbiertem und exportiertem.

die aktuellen Werte werden i.d.R. über ein 15 Minuten Mittel erhalten. Manche Länder sende jedoch nur stündlich ihre Daten.
Die Energie lässt sich dadurch durch Leistung * Zeit berechnen.

Eine Umrechnung in kWh bedeutet also ein Teilen durch Faktor 4 der Summe (wenn Stunde die Gruppierung ist)

## 1. Daten abfragen und speichern

Die Daten stehen über eine API zur Verfügung. Hierzu benötigt man einen api_key, welchen man mit einer Mail-Adresse beantragen kann.
Anschließend kann man anfragen an die API stellen. Hierzu benötigt man für jede Area den zugehörigen Code.
Diese antwortet mit CSV bzw manchmal gezipptem CSV.

Der Umgang hiermit wird erheblich durch eine bestehende Python-Bibliothek erleichtert.

Mit der entsoe-py Bibliothek können wir die Daten direkt als Pandas-DataFrame runterladen.
Diese stellt die folgenden Endpunkte bereit:

In [ ]:
import pandas as pd
from entsoe import EntsoePandasClient
import matplotlib.pyplot as plt
client = EntsoePandasClient(api_key='ae2ed060-c25c-4eea-8ae4-007712f95375')

object_methods = [method_name for method_name in dir(client)
                  if callable(getattr(client, method_name))]
# only public ones
list(filter(lambda k: not str.startswith(k,'_'), object_methods))

wir beschränken uns hierbei auf:
* die Auslastung: `query_load`,
* die generierte Energie: `query_generation`,
* die generierte Energie pro Kraftwerk: `query_generation_per_plant`
* die vorhandenen Kapazitäten: `query_installed_generation_capacity`
* diese gibt es auch noch pro Kraftwerk: `query_installed_generation_capacity_per_unit`
* außerdem gibt es noch die Übertragungsdaten zwischen den Ländern: `query_crossborder_flows`

Der Beginn der Datenaufzeichnungen im neuen Format der ENTSO-E ist der 1.1.2015 für Deutschland. Andere Länder haben erst später mit den Aufzeichnungen angefangen.

In [ ]:
# german data available since
begin = pd.Timestamp('20150101', tz='Europe/Berlin')

Schauen wir uns mal die Auslastung der letzten 5 Tage an

In [ ]:
from datetime import datetime, date, timedelta
from entsoe.mappings import PSRTYPE_MAPPINGS,NEIGHBOURS,Area
areas = pd.DataFrame([[e.name,e.value,e._tz,e._meaning] for e in Area])
load = pd.DataFrame()
country_code = 'DE'

today=datetime.strftime(datetime.now().date(),'%Y%m%d')
prev5=datetime.now().date()-timedelta(days=5)
prev5str=datetime.strftime(prev5,'%Y%m%d')

start = pd.Timestamp(prev5str, tz='Europe/Berlin')
end = pd.Timestamp(today, tz='Europe/Berlin')

load[country_code] = client.query_load(country_code, start=start,end=end)


In [ ]:
df = pd.DataFrame(load['DE'])

import plotly.express as px 
fig = px.line(df, x=df.index, y='DE')

fig.show()

Zu beachten ist hierbei, dass die Daten zum einen pro Land erfasst werden, zum anderen auch pro BiddingZone, also pro Marktgebiet.
So gibt es ebenfalls Daten für DE, DE_LU, DE_AT_LU, DE_50HERTZ, DE_AMPRION.

Die Unterscheidung muss für alle Datenwerte beachtet werden, was visualisiert werden soll.

Intern gibt es für diese Bereiche noch eine nicht lesbare ID. Die Übersetzung in die Marktgebiete wie sie auf der Webseite zu finden sind, erfolgt bereits in dem Python-Package

In [ ]:
[print (e.name) for e in Area] 

## Karte der Kraftwerke

Die Kraftwerke eines Marktgebietes kann man sich über die API runterladen:

In [ ]:
prod = client.query_installed_generation_capacity_per_unit('DE_AT_LU',start=start,end=end)
prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('BE',start=start,end=end)])
prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('DE_AMPRION',start=start,end=end)])
prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('DE_50HZ',start=start,end=end)])
prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('DE_LU',start=start,end=end)])
prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('FR',start=start,end=end)])
prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('CH',start=start,end=end)])
prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('AT',start=start,end=end)])
prod = pd.concat([prod,client.query_installed_generation_capacity_per_unit('GB',start=start,end=end)])
prod.head()

Allerdings sind hier keine Koordinaten enthalten. Die Koordinaten für "Konventionelle Kraftwerke" werden glücklicherweise von einer externen Webseite bereitgestellt:

Leider konnte ich keine Quelle finden, welche die Werte frei verfügbar nach EIC_CODE aufgeschlüsselt für Solar und Wind ebenfalls enthält.

In [ ]:
df = pd.read_csv('https://data.open-power-system-data.org/conventional_power_plants/latest/conventional_power_plants_EU.csv')
# uns interessieren nur Daten, welche auch Koordinaten und einen Key enthalten
df.dropna(axis=0,subset=['lon','lat','eic_code'],inplace=True)

Nun können wir durch einen join der Datasets eine Karte von Kraftwerken erstellen:

In [ ]:
df.index = df['eic_code']
joined = prod.join(df)[['Name','lon','lat','capacity','Production Type','country']]
joined.dropna(axis=0,subset=['lon','lat'],inplace=True)
joined.head()

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(joined, lat="lat", lon="lon", color='country',hover_name="Name",hover_data=["capacity",'Production Type'],zoom=3)
fig.update_layout(mapbox_style="open-street-map",margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

Die Karte kann man nach Ländern filtern und sich einzelne Punkte oder Länder ansehen.

Alternativ kann man die Karte farblich statt nach Ländern auch nach Kraftwerktyp visualisieren:

In [ ]:
import plotly.express as px
fig = px.scatter_mapbox(joined, lat="lat", lon="lon", color='Production Type',hover_name="Name",hover_data=["capacity",'Production Type'],zoom=3)
fig.update_layout(mapbox_style="carto-positron",margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [ ]:
# areas containing load content
valid_areas=[]
for e in Area:
    country_code = e.name
    try:
        load[country_code] = client.query_load(country_code, start=start,end=end)
        valid_areas.append(country_code)
    except Exception as e:
        pass
        #print('failed:',country_code, e)
        
valid_countries = list(filter(lambda x: len(x)<=2,valid_areas))
"['"+"','".join(valid_countries)+"']"

In [ ]:
import plotly.graph_objects as go
data= []
valid_countries=['DE','BE','NL','FR','CZ','PL','NO','IT','ES']
#for country_code in valid_countries:
for country_code in ['DE','BE','NL','FR','CZ','PL','NO','IT','ES']:
    load[country_code]=client.query_load(country_code, start=start,end=end)
    data.append(go.Scatter(x=load.index,y=load[country_code]/1000,name=country_code,   
        fill='tozeroy',
        fillcolor='rgba(26,0,65,0.1)',
        #stackgroup='one',
        #stackgaps='interpolate'
        ))

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets

textbox = widgets.Dropdown(
    description='Länder:   ',
    value=valid_countries[0],
    options=valid_countries,
    multiselect=True,
)

fig = go.FigureWidget(data=data)
fig.update_layout(
    title="Netzlast der Länder",
    xaxis_title="Zeit",
    yaxis_title="Leistung in MW",
    legend_title="Länder",
    font=dict(
        #family="Courier New, monospace",
        #size=18,
        #color="RebeccaPurple"
    ),)

import time
def response(change):
    with fig.batch_update():
        fig.data = []
        fig.add_trace(go.Scatter(x=load.index,y=load[textbox.value]/1000,name=textbox.value,
            fill='tozeroy',
            fillcolor='rgba(26,0,65,0.1)',
            #stackgroup='one',
            #stackgaps='interpolate'
            ))
        fig.update_layout(showlegend=True)
textbox.observe(response, names="value")

#fig.show()

widgets.VBox([widgets.HBox([textbox]),
              fig])

Die Verläufe sind in ihrer Bewegung der Netzauslastung sehr ähnlich. Je größer das Land, desto größer sind auch die Schwankungen.

Wenn man sich die Daten anguckt, sieht man, dass nicht alle Daten viertelstündlich, wie DE vorliegen. Spanien (ES) melde die Daten nur stündlich. Das muss bei der Speicherung beachtet werden.

Betrachten wir noch die Kapazitäten der Länder:

## Kapazität der Länder nach Jahren

In [ ]:
capacity = {}
start1 = pd.Timestamp('20150101', tz='Europe/Berlin')
end1 = pd.Timestamp('20200102', tz='Europe/Berlin')

for country_code in valid_countries:
    capacity[country_code] = client.query_installed_generation_capacity(country_code, start=start1,end=end1)
    

In [ ]:
from ipywidgets import widgets

country_code =list(capacity.keys())[0]

textbox = widgets.Dropdown(
    description='Länder:   ',
    value=country_code,
    options=list(capacity.keys()),
)
data= []
curDat = capacity[country_code]
for fuel in list(curDat.keys()):
        data.append(go.Scatter(x=curDat.index,y=curDat[fuel]/1000,name=fuel,   
    #fill='tozeroy',
    stackgroup='one',
    stackgaps='interpolate'
    ))

fig = go.FigureWidget(data=data)
fig.update_layout(
    title="Erzeugungskapazitäten der Länder, unterteilt nach Sorte",
    xaxis_title="Jahr",
    yaxis_title="Erzeugungskapazität in MW",
    legend_title="Sorten",
    font=dict(
        #family="Courier New, monospace",
        #size=18,
        #color="RebeccaPurple"
    ),)

def response(change):
    with fig.batch_update():
        fig.data = []
        curDat = capacity[textbox.value]
        for fuel in list(curDat.keys()):
            fig.add_trace(go.Scatter(x=curDat.index,y=curDat[fuel]/1000,name=fuel,   
                #fill='tozeroy',
                stackgroup='one',
                stackgaps='interpolate'
                ))
        fig.update_layout(showlegend=True)
textbox.observe(response, names="value")

#fig.show()

widgets.VBox([widgets.HBox([textbox]),
              fig])

Hier hat man leider auch nicht immer die selben verfügbaren Werte. Zur Speicherung in einer Datenbank muss man also das Schema dynamisch aufbauen oder vorher alle jemals verfügbaren Erzeugnisformen kennen.

Später möchte man in einer Visualisierung den einzelnen Erzeugnissen auch immer die selbe Farbe geben (Solar=gelb,Kohle=Schwarz usw), weshalb man auch hier mit fehlenden oder hinzukommenden Werten klar kommen muss.  

Holprig wird es erst bei den Stromerzeugungs-Daten:

## Stromerzeugung in Deutschland in aktueller Woche

In [ ]:
def replaceStr(string):
    '''
    Apache Spark gefallen einige Zeichen der Columns nicht.
    Diese werden hier ersetzt
    '''
    
    st = str.replace(string,')','')
    st = str.replace(st,'(','')
    st = str.replace(st,',','')
    st = str.replace(st,"'",'')
    st = st.strip()
    st = str.replace(st,' ','_')
    return st

Zunächst holen wir uns die Daten über die API:

In [ ]:
#gen=pd.DataFrame() # does not work with differing columns
gen={}
country='GR'
gen[country] = client.query_generation(country, start=start,end=end) # no data for DE
gen[country]['time']=gen[country].index
country='DE'
gen[country] = client.query_generation(country, start=start,end=end) # no data for DE
gen[country]['time']=gen[country].index

In [ ]:
gen['DE'].head()

In [ ]:
gen['GR'].head()

Man sieht also, dass die Daten unterschiedliche Strukturen haben. Nicht immer erfolgt die Aufschlüsselung nach Erzeugnissen (Actual Aggregated) und der zur Erzeugung benötigten Energie (actual Consumption)

Hierzu müssen die Daten über die Differenzen gebildet werden:

In [ ]:
#convert multiindex to single index
gen['DE'].columns = list(map(replaceStr, map(str,gen['DE'].columns)))
# unpivot data for visualization
g = gen['DE'].melt(id_vars=['time'],  var_name='kind', value_name='value')

In [ ]:
import plotly.express as px 
fig = px.line(g, x='time', y='value', color='kind',title='Stromerzeugnis in Deutschland der letzten 5 Tage in kWh')

fig.show()

In dieser Grafik kann man mit einem Doppelklick auf "Hydro Pumped Storage Actual Aggregated" und anschließendem einfach Klick "Hydro Pumped Storage Actual Consumption" sich zwei Series zusammen angucken.

Hier sieht man, dass der Pumpwasserspeicher tagsüber Strom erhält und nachts Strom in das Netz einspeist.


Für Griechenland erhält man diese Daten nicht aufgeschlüsselt, weshalb hier nur der eingespeiste Wert verfügbar ist.

In [ ]:
#convert multiindex to single index
gen['GR'].columns = list(map(replaceStr, map(str,gen['GR'].columns)))
# unpivot data for visualization
g = gen['GR'].melt(id_vars=['time'],  var_name='kind', value_name='value')

In [ ]:
import plotly.express as px 
fig = px.line(g, x='time', y='value', color='kind')
fig.update_layout(
    title='Stromerzeugnis in Griechenland der letzten 5 Tage in kWh',
    xaxis_title="Datum",
    yaxis_title="Erzeugnis in kWh",
    legend_title="Erzeugungsform",    
)
fig.show()

Damit man diese Daten nun unter einen Hut bekommt, muss man nun jeweils die Differenz zwischen Consumption und Aggregation erhalten und den MultiIndex gegebenenfalls umbenennen.

In [ ]:
def calcDiff(data, inplace=False):
    '''
    Berechnet jeweils Differenzen zwischen den zugehörigen zwei Spalten.
    '''
    if not inplace:
        dat = data.copy()
    else:
        dat=data
    for c in filter(lambda x:x.endswith('_Actual_Aggregated'), dat.columns):        
        new = str.replace(c,'_Actual_Aggregated','')
        dif = list(filter(lambda x: x.endswith('_Actual_Consumption') and x.startswith(new), dat.columns ))
        if len(dif) > 0:
            # wenn es beides gibt wird die Differenz gebildet
            print(dif[0])
            dat[new]=dat[c]-dat[dif[0]]
            del dat[c]
            del dat[dif[0]]
        else:
            # sonst wird direkt 
            dat[new]=dat[c]
            del dat[c]
    for c in filter(lambda x:x.endswith('_Actual_Consumption'), dat.columns):
        # wenn es nur Verbrauch aber kein Erzeugnis gibt, mach negativ
        new = str.replace(c,'_Actual_Consumption','')
        dat[new]=-dat[c]
        del dat[c]
    return dat

In [ ]:
data = gen['DE']

In [ ]:
data.columns = list(map(replaceStr, map(str,data.columns)))
data.fillna(0, inplace=True)
# calculate difference betweeen agg and consumption
data=calcDiff(data)

Nun haben wir einheitliche Namen der Erzeugnisformen:

In [ ]:
data.columns

In Deutschland erhalten wir nun die Netto Erzeugnisse. Das erzeugt für den Pumpspeicher negative Werte, was die Interpretation auch erleichtert.

Im Area-Plot ist die Visualisierung trotzdem korrekt und stellt die Erzeugung nach Erzeugungsform dar.

In [ ]:
g = data.melt(id_vars=['time'],  var_name='kind', value_name='value')
fig = px.area(g, x='time', y='value', color='kind')
fig.update_layout(
    title='Stromerzeugnis in Deutschland der letzten 5 Tage in kWh',
    xaxis_title="Datum",
    hovermode="closest",
    yaxis_title="Erzeugnis in kWh",
    legend_title="Erzeugungsform",    
)
fig.show()

# Auswahl des Landes über eine Chloropleth Karte

Die Auswahl eines Landes soll durch Anklicken des Gebietes ermöglicht werden.
Eine mögliche Visualisierung ist auch, eine Zeit auszuwählen, zu der die Karten nach aktuellem Load visualisiert werden.
Das ist allerdings nur begrenzt sinnvoll, wenn man die Gebietsgröße nicht mit einbezieht.

Alternativ könnte man den Wert relativ zur Maximallast betrachten.
Das sei hier aber nur erwähnt und wurde aufgrund des Aufwands nicht durchgeführt

In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
countries = ['DE','FR','GR','PL','LU','IT']

with open("europe.geo.json", "r", encoding="utf-8") as f:
    geo = json.load(f)    

#geo['features'][0]
df = pd.DataFrame()
df['countries']=countries
df['values']=list(map(lambda x: ord(x[0]),countries))

import plotly.express as px

fig = px.choropleth_mapbox(df, geojson=geo, locations="countries", color='values',
                           color_continuous_scale="Jet",
                           featureidkey="properties.iso_a2",
                           range_color=(65, 80),
                           mapbox_style="carto-positron", # open-street-map
                           zoom=3, center = {"lat": 50.0902, "lon": 10.7129},
                           opacity=0.5,
                           labels={'values':'Werte'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

## Wie hat sich die Stromförderung durch Corona verändert?

Problematik: Daten liegen nur viertelstündlich vor. Zum sinnvollen Gruppieren (nach Monat) wird eine Datenspeicherung benötigt und Zugriff auf die Historiendaten.

Es ist möglich die Daten in Parquet zu speichern.
Ein andere sinnvolle Möglichkeit erscheint SQLite zu sein, da man es recht leicht ansprechen kann und dank in-memory-Datenbanken auch ordentlich Performance in einem nicht-verteilten System erhält.
Da lediglich viertelstündlich Daten gespeichert werden und die Datenmenge ungefähr 60 Länder * ~ 16 Spalten + 60 Länder * ~50 Kraftwerke also ganz grob 3600 Werte enthält, welche in einem relationalen Format gespeichert werden, kommt man hier mit konventionellen RDBMS gut davon.

Spannend wären bspw Abweichungen von den Vormonaten.

## Speicherung der Daten

Zur Speicherung der Daten müssen diese Paket-weise von der API runtergeladen werden und in einem sinnvollen Format abgespeichert werden.

Nach einem ersten Ansatz, welcher pro Land eine eigene Tabelle vorsah, wurde nun das folgende Datenbank-Schema gewählt:

![ER-Diagramm](diagrams-ENTSO-E.png)

Hier gibt es nur noch für die Erzeugungen nach Kraftwerk eine eigene Tabelle pro Land.
Wobei diese vermutlich auch sinnvoller als unpivot Tabelle behandelt werden sollte.

Zum Herunterladen der Daten wurde eine Klasse angelegt, welche die Paketierung übernimmt und sowohl die Speicherung in eine SQLite-Tabelle übernimmt, wie auch in Parquet übernimmt.

Hierbei wurde viel mit der Speicherung der Datumswerte und einer Partitionierung der Parquet-Dateien experimentiert, welche Performance-technisch auf dem Programmier-Laptop nichts änderten

Die Speicherung der Zeitwerte läuft nativ schon sehr gut, durch ein Abspeichern des TimeStamps.

Der TimeStamp lässt sich in SQLite zur Gruppierung mit der SQLite-Funktion strftime manipulieren.
Durch `strftime("%Y-%m-%d %H:00:00", "index") as time` erhält man den zur Stunde abgeschnittenen Wert.
Mit `strftime("%Y-%m-%d", "index") as time` respektive den Wert für ein Zusammenfassen nach Tagen

Eine Gruppierung nach `time` und aggregation mit sum bzw avg bringt die gewünschten Ergebnisse.

Mit Apache Spark und Parquet läuft das ziemlich analog.

Den TimeStamp kann man sich über die spark.sql.function date_trunc abschneiden lassen. Hier kann man direkt sagen, wo abgeschnitten werden soll: `spark_df.withColumn("time", date_trunc('day',"time"))`

## SPARK

Der Einfachheithalber wurden die Daten bereits herunter geladen und mithilfe von Parquet abgespeichert

In [ ]:
import findspark

findspark.init()
from pyspark import SparkConf
from pyspark.sql import SparkSession
import pandas as pd

conf = SparkConf().setAppName('entsoe').setMaster('local')
spark = SparkSession.builder.config(conf=conf).getOrCreate()
spark

In [ ]:
loadDE = spark.read.parquet('data/DE/query_load')

In [ ]:
beg=loadDE.select('time').rdd.min()
end=loadDE.select('time').rdd.max()

print('Datensequenz von {} bis {}'.format(beg[0],end[0]))

In [ ]:
from pyspark.sql.functions import date_trunc
groupTime='month'

loadBE = spark.read.parquet('data/BE/query_load').withColumn(groupTime, date_trunc(groupTime,"time"))
floadBE = spark.read.parquet('data/BE/query_load_forecast').withColumn(groupTime, date_trunc(groupTime,"time"))

In [ ]:
%time pl=loadBE.groupby(groupTime).avg('0').toPandas()
%time pf=floadBE.groupby(groupTime).avg('0').toPandas()

In [ ]:
import plotly.express as px


pl.sort_values(groupTime,inplace=True)
pf.sort_values(groupTime,inplace=True)
pf['forecast']=pf['avg(0)']
pf['actual']=pl['avg(0)']
fig = px.line(pf, x=groupTime, y=['actual','forecast'], title='Auslastung Belgien')
fig.update_layout( xaxis_title='Datum',
                   yaxis_title='Durchsatz in kW gemittelt pro Stunde')
fig.show()

Zum Vergleich das ganze auch in SQLite:

In [ ]:
import sqlite3 as sql
from contextlib import closing
country = 'BE'
selectString='strftime("%Y-%m-01", "index") as time, avg("0") as value'
groupString='strftime("%Y-%m-01", "time")'
with closing(sql.connect('data/entsoe2.db')) as conn:
    query = f"select {selectString} from {country}_query_load group by {groupString}"
    %time load = pd.read_sql_query(query,conn,index_col='time')
    query = f"select {selectString} from {country}_query_load_forecast group by {groupString}"
    %time forecast = pd.read_sql_query(query,conn,index_col='time')

In [ ]:
load['forecast']=forecast['value']

fig = px.line(load, x=load.index, y=['value','forecast'], title='Auslastung Belgien')
fig.update_layout( xaxis_title='Datum',
                   yaxis_title='Durchsatz in kW gemittelt pro Stunde')
fig.show()

## Fluss zu benachbarten Ländern


* Karte mit Änderungsflüssen
* wunderschön wäre eine Animation über die Zeit, mit Darstellung der Förderung durch Farben

Es kann festgestellt werden wie der Fluss zu benachbarten Ländern ist.

Hierzu kann beim Anklicken eines Landes der Netto-Transfer (Exportiert-importiert) berechnet werden.
Die Daten müssen hierzu in ein geeignetes Format gebracht werden (neighbours Tabelle an crossboarders joinen..)
Eigentlich müsste an jede Beziehung der Neighbours-Tabelle eine Series gejoined werden ->
    realisieren durch crossboarder-Tabelle, welche alle nachbarschaftsbeziehungen als Key enthält

In [ ]:
def selectBuilder(n):
    res =''
    for x in n:
        fr =x.split('.')[0]
        to =x.split('.')[1]
        # export - import
        res += f'sum("{fr}.{to}"-"{to}.{fr}") as diff_{to}'
        res += ','
    return res
        
def neighbours(fr):
    nei=[]
    for x in columns:
        sp = x.split('.')
        if sp[0]==fr:
            nei.append(x)  
            #nei.append(sp[1]+'.'+sp[0])
    return nei

In [ ]:
country = 'ES'

with closing(sql.connect('data/entsoe2.db')) as conn:
    query = 'select * from query_crossborder_flows where 0=1'
    columns= pd.read_sql_query(query,conn).columns
    nei = neighbours(country)
    n = selectBuilder(nei)
    selectString='strftime("%Y-%m-%d %H:00:00", "index") as time,'
    groupString='strftime("%Y-%m-%d  %H:00:00", "index")'
    query = f'select {selectString}{n} "index" from query_crossborder_flows group by {groupString}'
    neigh = pd.read_sql(query,conn,index_col='index')

In [ ]:
fig = px.line(neigh, x=neigh.index, y=['diff_FR','diff_PT'], title='Netto Export an Nachbarn von Spanien')
fig.update_layout( xaxis_title='Datum',
                   yaxis_title='Export - Import an Nachbarn')
fig.show()

Zu erkennen ist, dass Spanien von Portugal importiert und mehr nach Frankreich exportiert.

## Sankey-Diagramm über Gesamt-Generierte Daten und Flüsse zwischen den Ländern

Über die Jahre kann man dann eine Art Zustandsdiagramm machen, welcher Staat wie autark ist und wer stets Strom zugeführt bekommen muss

# Wie gut ist die Vorhersage der erzeugten Daten? Können wir etwas besseres?

ENTSO-E schreibt:
> The day-ahead forecast is calculated (estimated) on the historic load profile on similar days, taking into account the variables that affect electricity demand, such as weather conditions, climate and socioeconomic factors.

Vielleicht lässt sich mit Machine-Learning ein besseres Modell finden.

Problematik: zyklische Trainingsdaten, weniger Daten (Wetter, Klima und Sozioökonomische Daten fehlen)

Abweichung der Schätzungen vergleichen